In [1]:
!pip install koreanize-matplotlib

In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import Day, Hour, Minute, Second


train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')


print('train shape:',train.shape)
print('test shape:',test.shape)
train.head()

train shape: (10886, 12)
test shape: (6493, 9)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [2]:
sub=pd.read_csv('/content/sampleSubmission.csv')
print('submission shape:', sub.shape)
sub.head()

submission shape: (6493, 2)


,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0


In [3]:
# Prophet Library
from prophet import Prophet

# Prophet 시각화 도구
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.plot import add_changepoints_to_plot

In [4]:
y_train_prophet_df = train[['datetime','count']]
y_train_prophet_df.columns = ['ds', 'y']
y_train_prophet_df.head()

,ds,y
0,2011-01-01 00:00:00,16
1,2011-01-01 01:00:00,40
2,2011-01-01 02:00:00,32
3,2011-01-01 03:00:00,13
4,2011-01-01 04:00:00,1


In [5]:
x_col = [i for i in train.columns if i not in ['datetime','count']]
x_train_prophet_df = train[x_col]
x_train_prophet_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered
0,1,0,0,1,9.84,14.395,81,0.0,3,13
1,1,0,0,1,9.02,13.635,80,0.0,8,32
2,1,0,0,1,9.02,13.635,80,0.0,5,27
3,1,0,0,1,9.84,14.395,75,0.0,3,10
4,1,0,0,1,9.84,14.395,75,0.0,0,1


In [6]:
yx_train_prophet_df = pd.concat(
                                [y_train_prophet_df, x_train_prophet_df], axis=1
                              )

In [7]:
# Applying Prophet Model

fit_default_prophet = Prophet(
                          # 1) linear, nonlinear
                          growth='linear',

                          # 2) Trend
                          changepoints=None, # CP가 발생하는 시점들의 list ['2012-01-01']
                          n_changepoints=25, # CP의 수
                          changepoint_range=0.8, # CP의 기존 데이터 수 대비 최대 비율
                          changepoint_prior_scale=0.05, # CP 추정 민감도로 높을수록 민감

                          # 3) Seasonality
                          seasonality_mode='multiplicative', # 계절성 모델: 'additive' or 'multiplicative'
                          seasonality_prior_scale=10.0, # 계절성 추정 민감도로 높을수록 민감
                          yearly_seasonality='auto', # 연계절성
                          weekly_seasonality='auto',# 월계절성
                          daily_seasonality='auto', #일계절성

                          # 4) Holiday
                          holidays=None, # 휴일 또는 이벤트 시점
                          holidays_prior_scale=10.0, # 휴일 추정 민감도로 높을수록 민감

                          # 5) Others
                          interval_width=0.8, # 추세 예측 정확도 구간범위
                          mcmc_samples=0, # 계절성 예측 정확도 제어
                          )

In [8]:
yx_train_prophet_df

,ds,y,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered
0,2011-01-01 00:00:00,16,1,0,0,1,9.84,14.395,81,0.0000,3,13
1,2011-01-01 01:00:00,40,1,0,0,1,9.02,13.635,80,0.0000,8,32
2,2011-01-01 02:00:00,32,1,0,0,1,9.02,13.635,80,0.0000,5,27
3,2011-01-01 03:00:00,13,1,0,0,1,9.84,14.395,75,0.0000,3,10
4,2011-01-01 04:00:00,1,1,0,0,1,9.84,14.395,75,0.0000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,336,4,0,1,1,15.58,19.695,50,26.0027,7,329
10882,2012-12-19 20:00:00,241,4,0,1,1,14.76,17.425,57,15.0013,10,231
10883,2012-12-19 21:00:00,168,4,0,1,1,13.94,15.910,61,15.0013,4,164
10884,2012-12-19 22:00:00,129,4,0,1,1,13.94,17.425,61,6.0032,12,117


In [9]:
# model 학습
fit_reg1_prophet = fit_default_prophet.fit(yx_train_prophet_df)

# Frequency에 대한 사전 df를 만듦
forecast = fit_reg1_prophet.make_future_dataframe(freq='H',periods=sub.shape[0])

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmprtvs8lsx/jhesd953.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmprtvs8lsx/39q4lzqk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=49607', 'data', 'file=/tmp/tmprtvs8lsx/jhesd953.json', 'init=/tmp/tmprtvs8lsx/39q4lzqk.json', 'output', 'file=/tmp/tmprtvs8lsx/prophet_modelk3vp2wv8/prophet_model-20230720060913.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
06:09:13 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
06:09:15 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [10]:
fit_reg1_prophet

In [11]:
# predict
pred_reg1_prophet = fit_reg1_prophet.predict(forecast)

In [12]:
pred_train_reg1_prophet = np.ravel(pred_reg1_prophet.loc[:yx_train_prophet_df.shape[0]-1, ['yhat']])
pred_test_reg1_prophet = np.ravel(pred_reg1_prophet.loc[yx_train_prophet_df.shape[0]:, ['yhat']])

In [13]:
sub['count'] = pred_test_reg1_prophet
sub['count'] = sub['count'].apply(lambda x:round(x,0))
sub.loc[sub['count']<0,'count'] = 0
sub.head()

,datetime,count
0,2011-01-20 00:00:00,95.0
1,2011-01-20 01:00:00,75.0
2,2011-01-20 02:00:00,26.0
3,2011-01-20 03:00:00,0.0
4,2011-01-20 04:00:00,0.0


In [14]:
sub.to_csv("kaggle_time_04.csv", index = False)